In [1]:
from transformers import BertTokenizerFast, BertModel
tokenizer_bert = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model_bert = BertModel.from_pretrained("kykim/bert-kor-base")

print(model_bert.train())
print(model_bert.__dict__)

c:\Users\mnh51\anaconda3\envs\tour2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'training': False, '_parameters': OrderedDict(), '_buffers': OrderedDict(), '_non_persistent_buffers_set': set(), '_backward_hooks': OrderedDict(), '_is_full_backward_hook': None, '_forward_hooks': OrderedDict(), '_forward_pre_hooks': OrderedDict(), '_state_dict_hooks': OrderedDict(), '_load_state_dict_pre_hooks': OrderedDict(), '_load_state_dict_post_hooks': OrderedDict(), '_modules': OrderedDict([('embeddings', BertEmbeddings(
  (word_embeddings): Embedding(42000, 768, padding_idx=0)
  (position_embeddings): Embedding(512, 768)
  (token_type_embeddings): Embedding(2, 768)
  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
  (dropout): Dropout(p=0.1, inplace=False)
)), ('encoder', BertEncoder(
  (layer): ModuleList(
    (0): BertLayer(
      (attention): BertAttention(
        (self): BertSelfAttention(
          (query): Linear(in_features=768, out_features=768, bias=True)
          (key): Linear(in_features=768, out_features=768, bias=True)
          (value): Line

In [4]:
inputs = tokenizer_bert("안녕하세요. 어 그러니까, 대한민국의 수도는 서울입니다. 정리하자면, 캠핑장이 위치해있는 곳은 가평입니다. [PAD][PAD]", return_tensors='pt')
print(inputs['input_ids'].shape)
print(inputs['input_ids'])
print(inputs['attention_mask'])
print(inputs['token_type_ids'])
output = model_bert(
    input_ids=inputs['input_ids'][:,:512],
    attention_mask=inputs['attention_mask'][:,:512],
    token_type_ids=inputs['token_type_ids'][:,:512])
print(output.__dict__)
print(output.last_hidden_state.shape)
print(output.pooler_output.shape)

torch.Size([1, 26])
tensor([[    2, 16453,  2016,  5590, 18421,  2014, 23589, 14414,  8034, 14328,
         13992,  2016, 14748, 19686,  2014, 33474,  8018, 21457, 14140, 17045,
         26827, 13992,  2016,     0,     0,     3]])
tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1]])
tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0]])
{'last_hidden_state': tensor([[[ 0.3753, -0.1631,  0.3628,  ...,  0.1089, -1.4743,  0.3935],
         [ 0.4944,  0.2368, -0.2590,  ...,  0.1082, -1.4293, -0.0469],
         [ 0.4803, -0.3952,  0.2008,  ...,  0.9918, -1.9435, -0.3293],
         ...,
         [ 1.6074,  0.6423,  0.0538,  ..., -0.3423, -0.9340, -1.1100],
         [ 1.1927,  0.1356, -0.5316,  ..., -0.6882, -0.4671, -0.5094],
         [ 1.3811,  0.1645,  0.3409,  ..., -0.1788, -1.7024, -0.7521]]],
       grad_fn=<NativeLayerNormBackward0>), 'pooler_output': tensor([[ 0.3681,  0.0123, -0.9371,  0.9423, -0.

In [2]:
import pandas as pd
import torch

all_df = pd.read_csv('./train.csv')
print(type(all_df))
all_df['bert_emb'] = ""

i=0
for ov in all_df['overview']:
    inputs = tokenizer_bert(ov, return_tensors='pt')
    if len(inputs['input_ids'][0]) > 512:
        output = model_bert(
            input_ids=inputs['input_ids'][:,:512],
            attention_mask=inputs['attention_mask'][:,:512],
            token_type_ids=inputs['token_type_ids'][:,:512]
        ).pooler_output
    else:
        output = model_bert(
            input_ids=inputs['input_ids'][:,:512],
            attention_mask=inputs['attention_mask'][:,:512],
            token_type_ids=inputs['token_type_ids'][:,:512]
        ).pooler_output

    out = output.cpu().detach().numpy()
    all_df['bert_emb'][i] = out[0]
    i+=1

    if i%1000 == 0:
        print('i : ', i)

all_df.to_csv('./train_bert.csv')

<class 'pandas.core.frame.DataFrame'>


Token indices sequence length is longer than the specified maximum sequence length for this model (1125 > 512). Running this sequence through the model will result in indexing errors


i :  1000
i :  2000
i :  3000
i :  4000
i :  5000
i :  6000
i :  7000
i :  8000
i :  9000
i :  10000
i :  11000
i :  12000
i :  13000
i :  14000
i :  15000
i :  16000


In [3]:
import pandas as pd
import torch

from transformers import BertTokenizerFast, BertModel
tokenizer_bert = BertTokenizerFast.from_pretrained("kykim/bert-kor-base")
model_bert = BertModel.from_pretrained("kykim/bert-kor-base")

all_df = pd.read_csv('./test.csv')
print(type(all_df))
all_df['bert_emb'] = ""

i=0
for ov in all_df['overview']:
    inputs = tokenizer_bert(ov, return_tensors='pt')
    if len(inputs['input_ids'][0]) > 512:
        output = model_bert(
            input_ids=inputs['input_ids'][:,:512],
            attention_mask=inputs['attention_mask'][:,:512],
            token_type_ids=inputs['token_type_ids'][:,:512]
        ).pooler_output
    else:
        output = model_bert(
            input_ids=inputs['input_ids'][:,:512],
            attention_mask=inputs['attention_mask'][:,:512],
            token_type_ids=inputs['token_type_ids'][:,:512]
        ).pooler_output

    out = output.cpu().detach().numpy()
    all_df['bert_emb'][i] = out[0]
    i+=1

all_df.to_csv('./test_bert.csv')

<class 'pandas.core.frame.DataFrame'>


Token indices sequence length is longer than the specified maximum sequence length for this model (545 > 512). Running this sequence through the model will result in indexing errors
